In [1]:
import pandas as pd 
import geopy.distance
import os
import json
import pickle
import uuid
import warnings
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime
import boto3
import numpy as np
import pandas as pd
from dateutil import relativedelta
from scipy.stats import randint as sp_randint
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import holidays
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import xgboost
from sklearn.cluster import KMeans

In [2]:
cd ..


/Users/nathantoubiana/Documents/Codes/Kaggle


In [3]:
cd all

/Users/nathantoubiana/Documents/Codes/Kaggle/all


In [4]:
df=pd.read_csv('train.csv')

In [5]:
def dist(lat1,long1,lat2,long2):
    try:
        coords_1 = (long1,lat1)
        coords_2 = (long2,lat2)

        return(geopy.distance.vincenty(coords_1, coords_2).km)
    except:
        return 'bug'

In [6]:
def lat_dist(lat1,long1,lat2,long2):
    return dist(lat1,long1,lat2,long1)

In [7]:
def long_dist(lat1,long1,lat2,long2):
    return dist(lat1,long1,lat1,long2)

In [8]:
df['distance']=df.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

/Users/nathantoubiana/anaconda3/lib/python3.6/site-packages/geopy/point.py:81: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  UserWarning)


In [9]:
df=df[df.distance!='bug']

In [10]:
df['latdistance']=df.apply(lambda x: lat_dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [11]:
df['longdistance']=df.apply(lambda x: long_dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [12]:
df=df[df.distance!='bug']

In [13]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,latdistance,longdistance
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.412061,0.280909,0.301449
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,4.64623,2.173992,4.104950
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.0048,0.329929,0.949115
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,0.914154,0.768510,0.495205
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.36616,0.485836,1.276794


In [200]:
y = df['fare_amount'].dropna(axis=0, how='any').as_matrix()

In [202]:
X.shape

(9999545, 6)

In [201]:
X = (df[[ 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance']]
                 .dropna(axis=0, how='any')
                 .as_matrix())

In [203]:
clf = RandomForestRegressor(max_depth=15, max_features=5, n_estimators=100,n_jobs=-1)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=400)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [204]:
clf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [205]:
df_test=pd.read_csv('test.csv')

In [206]:
def dist(lat1,long1,lat2,long2):
    try:
        coords_1 = (long1,lat1)
        coords_2 = (long2,lat2)

        return(geopy.distance.vincenty(coords_1, coords_2).km)
    except:
        return 'bug'

In [207]:
df_test['distance']=df_test.apply(lambda x: dist(x.pickup_latitude,x.pickup_longitude,x.dropoff_latitude,x.dropoff_longitude),axis=1)

In [208]:
X = (df_test[['pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count','distance']]
                 ._get_numeric_data()
                 .as_matrix())

In [209]:
predicted = clf.predict(X)



In [210]:
df_test['preds']=predicted

In [211]:
df_test_keys=df_test[['key','preds']]

In [212]:
df_final=pd.read_csv('sample_submission.csv')

In [213]:
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']

In [214]:
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)

In [215]:
s = pickle.dumps(clf)
joblib.dump(clf, 'baseline_model.pkl')

['baseline_model.pkl']

MORE FEATURES

In [14]:
df.shape

(55421320, 11)

In [15]:
df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

In [16]:
def transform(data):
    # Extract date attributes and then drop the pickup_datetime column
    data['hour'] = data['pickup_datetime'].dt.hour
    data['day'] = data['pickup_datetime'].dt.day
    data['month'] = data['pickup_datetime'].dt.month
    data['year'] = data['pickup_datetime'].dt.year
    data['weekday']=data['pickup_datetime'].dt.weekday
    return data

In [17]:
df1=transform(df)

In [18]:
df1.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,latdistance,longdistance,hour,day,month,year,weekday
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:00,-73.844311,40.721319,-73.841610,40.712278,1,0.412061,0.280909,0.301449,17,15,6,2009,0
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:00,-74.016048,40.711303,-73.979268,40.782004,1,4.64623,2.173992,4.104950,16,5,1,2010,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,1.0048,0.329929,0.949115,0,18,8,2011,3
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:00,-73.987130,40.733143,-73.991567,40.758092,1,0.914154,0.768510,0.495205,4,21,4,2012,5
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,1.36616,0.485836,1.276794,7,9,3,2010,1


In [142]:
cd ..

/Users/nathantoubiana/Documents/Codes/Kaggle


In [143]:
cd kaggle

/Users/nathantoubiana/Documents/Codes/Kaggle/kaggle


In [9]:
from process import Process

In [141]:
#df1.to_csv('train_with_features.csv')
#loading data with precomputed distances
dfcut=pd.read_csv('train_with_features.csv',nrows=10000000)

In [144]:
#removing outliers
data=Process().remove_outliers(dfcut)

2018-08-10 17:53:17,895 - process - Removing outliers


In [145]:
data=Process().split_dates(data)

2018-08-10 17:53:21,534 - process - Extracting date info


In [146]:
data=Process().assign_clusters(data)

2018-08-10 17:53:30,127 - process - Fetching kmeans model
2018-08-10 17:53:30,286 - process - Assigning clusters


In [147]:
data=Process().add_day_density(data)

2018-08-10 17:54:39,097 - process - Adding day density


In [148]:
data=Process().add_holidays(data)

2018-08-10 17:54:47,604 - process - Adding holidays


In [149]:
pickup_density=data[['pickup_cluster']].groupby(data.pickup_cluster).count().add_suffix('_Count').reset_index()
dropoff_density=data[['dropoff_cluster']].groupby(data.dropoff_cluster).count().add_suffix('_Count').reset_index()

In [150]:
data=data.merge(pickup_density, how='left', on='pickup_cluster')
data=data.merge(dropoff_density, how='left', on='dropoff_cluster')
data['pickup_density'] = data.pickup_cluster_Count
data['dropoff_density'] = data.dropoff_cluster_Count

In [151]:
data = pd.get_dummies(data[['fare_amount','pickup_longitude',
                                      'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                                      'passenger_count', 'distance', 'year', 'month', 'day', 'hour',
                                      'weekday', 'latdistance', 'longdistance', 'holiday', 'day_density',
                                      'pickup_cluster', 'dropoff_cluster','pickup_density','dropoff_density']])

In [123]:
data.to_csv('process.csv')

In [125]:
dfcut=pd.read_csv('process.csv',nrows=5000000)

KeyboardInterrupt: 

In [152]:
y = data['fare_amount'].dropna(axis=0, how='any').as_matrix()

In [153]:
X = (data[[ 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'distance', 'year', 'month',
       'day', 'hour', 'weekday', 'latdistance', 'longdistance', 'holiday',
       'day_density', 'pickup_cluster_0', 'pickup_cluster_1',
       'pickup_cluster_2', 'pickup_cluster_3', 'pickup_cluster_4',
       'pickup_cluster_5', 'pickup_cluster_6', 'pickup_cluster_7',
       'pickup_cluster_8', 'pickup_cluster_9', 'dropoff_cluster_0',
       'dropoff_cluster_1', 'dropoff_cluster_2', 'dropoff_cluster_3',
       'dropoff_cluster_4', 'dropoff_cluster_5', 'dropoff_cluster_6',
       'dropoff_cluster_7', 'dropoff_cluster_8', 'dropoff_cluster_9','pickup_density','dropoff_density']]
                 .dropna(axis=0, how='any')
                 .as_matrix())

In [154]:
y.shape

(9790787,)

In [155]:
X.shape

(9790787, 37)

In [54]:

#TO DO: 

#add holidays
#retrain with larger max depth
#add features about location zones (kmeans?)
#remove absurd outliers (water, etc)
#try radius NN
#try ANNOY?
#try ensembles of above models? add meta learning?


In [22]:
clf = xgboost.XGBRegressor(max_depth=22, max_features=6, n_estimators=100,n_jobs=-1)
#make max depth larger : 20

In [23]:
#4:30
#->1h
clf.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=22, max_features=6, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [67]:
FI=clf.feature_importances_

In [48]:
cd ..

/Users/nathantoubiana/Documents/Codes/Kaggle


In [49]:
cd kaggle

/Users/nathantoubiana/Documents/Codes/Kaggle/kaggle


In [47]:
df_test=pd.read_csv('test.csv')

In [50]:
df_test=Process().assign_clusters(df_test)

2018-08-10 17:32:18,777 - process - Fetching kmeans model
2018-08-10 17:32:18,785 - process - Assigning clusters


In [51]:
df_test=Process().split_dates(df_test)

2018-08-10 17:32:18,932 - process - Extracting date info


In [52]:
        def dist(lat1, long1, lat2, long2):
            try:
                coords_1 = (long1, lat1)
                coords_2 = (long2, lat2)

                return (geopy.distance.vincenty(coords_1, coords_2).km)
            except:
                return 'bug'

        def lat_dist(lat1, long1, lat2, long2):
            return dist(lat1, long1, lat2, long1)

        def long_dist(lat1, long1, lat2, long2):
            return dist(lat1, long1, lat1, long2)

        df_test['distance'] = df_test.apply(
            lambda x: dist(x.pickup_latitude, x.pickup_longitude, x.dropoff_latitude, x.dropoff_longitude), axis=1)
        df_test['latdistance'] = df_test.apply(
            lambda x: lat_dist(x.pickup_latitude, x.pickup_longitude, x.dropoff_latitude, x.dropoff_longitude), axis=1)
        df_test['longdistance'] = df_test.apply(
            lambda x: long_dist(x.pickup_latitude, x.pickup_longitude, x.dropoff_latitude, x.dropoff_longitude), axis=1)
        

In [53]:
df_test=Process().assign_clusters(df_test)

2018-08-10 17:32:23,177 - process - Fetching kmeans model
2018-08-10 17:32:23,192 - process - Assigning clusters


In [54]:
df_test=Process().add_day_density(df_test)

2018-08-10 17:32:23,294 - process - Adding day density


In [55]:
df_test=Process().add_holidays(df_test)

2018-08-10 17:32:23,313 - process - Adding holidays


In [56]:
df_test=df_test.merge(pickup_density, how='left', on='pickup_cluster')
df_test=df_test.merge(dropoff_density, how='left', on='dropoff_cluster')
df_test['pickup_density'] = df_test.pickup_cluster_Count
df_test['dropoff_density'] = df_test.dropoff_cluster_Count

In [57]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_cluster,dropoff_cluster,hour,...,longdistance,pickup_datetime_2,Unnamed: 0,pickup_datetime_2_Count,day_density,holiday,pickup_cluster_Count,dropoff_cluster_Count,pickup_density,dropoff_density
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:00,-73.973320,40.763805,-73.981430,40.743835,1,9,9,13,...,0.905146,2015-01-27,2217,125,125,False,388696,376897,388696,376897
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:00,-73.986862,40.719383,-73.998886,40.739201,1,0,0,13,...,1.341968,2015-01-27,2217,125,125,False,287873,258382,287873,258382
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:00,-73.982524,40.751260,-73.979654,40.746139,1,9,9,11,...,0.320315,2011-10-08,1010,462,462,False,388696,376897,388696,376897
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:00,-73.981160,40.767807,-73.990448,40.751635,1,9,9,21,...,1.036616,2012-12-01,1430,554,554,False,388696,376897,388696,376897
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:00,-73.966046,40.789775,-73.988565,40.744427,1,6,9,21,...,2.513301,2012-12-01,1430,554,554,False,216721,376897,216721,376897


In [58]:
df_test_1 = pd.get_dummies(df_test[['pickup_longitude',
                                      'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                                      'passenger_count', 'distance', 'year', 'month', 'day', 'hour',
                                      'weekday', 'latdistance', 'longdistance', 'holiday', 'day_density',
                                      'pickup_cluster', 'dropoff_cluster','pickup_density','dropoff_density']])

In [60]:
df_test_1['pickup_cluster_7']=0
df_test_1['dropoff_cluster_7']=0


In [61]:
X = (df_test_1[[ 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'passenger_count', 'distance', 'year', 'month',
       'day', 'hour', 'weekday', 'latdistance', 'longdistance', 'holiday',
       'day_density', 'pickup_cluster_0', 'pickup_cluster_1',
       'pickup_cluster_2', 'pickup_cluster_3', 'pickup_cluster_4',
       'pickup_cluster_5', 'pickup_cluster_6', 'pickup_cluster_7',
       'pickup_cluster_8', 'pickup_cluster_9', 'dropoff_cluster_0',
       'dropoff_cluster_1', 'dropoff_cluster_2', 'dropoff_cluster_3',
       'dropoff_cluster_4', 'dropoff_cluster_5', 'dropoff_cluster_6',
       'dropoff_cluster_7', 'dropoff_cluster_8', 'dropoff_cluster_9','pickup_density','dropoff_density']]
                 ._get_numeric_data()
                 .as_matrix())

In [62]:
X.shape

(9914, 37)

In [63]:
predicted = clf.predict(X)



In [64]:
cd ..

/Users/nathantoubiana/Documents/Codes/Kaggle


In [65]:
cd all

/Users/nathantoubiana/Documents/Codes/Kaggle/all


In [66]:
df_test['preds']=predicted
df_test_keys=df_test[['key','preds']]
df_final=pd.read_csv('sample_submission.csv')
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)

In [182]:
s = pickle.dumps(clf)
joblib.dump(clf, 'xgb3_with_density_clusters_dummy_densities.pkl')

['xgb3_with_density_clusters_dummy_densities.pkl']

In [30]:
joblib.load('xgb3_with_density.pkl')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=20, max_features=5, min_child_weight=1, missing=nan,
       n_estimators=100, n_jobs=-1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [107]:
min(predicted)

2.8921134

In [68]:
from sklearn.preprocessing import MinMaxScaler

In [174]:
X_scaled_fi[:,:15].shape

(9914, 15)

In [98]:
#test
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled=scaler.transform(X)
X_scaled_fi=FI*X_scaled

In [156]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled_train=scaler.transform(X)
X_scaled_fi_train=FI*X_scaled_train

In [157]:
X_scaled_fi_train.shape

(9790787, 37)

In [158]:
y.shape

(9790787,)

In [159]:
from sklearn.neighbors import KNeighborsRegressor

In [178]:
neigh = KNeighborsRegressor(n_neighbors=3,n_jobs=-1)

In [179]:
neigh.fit(X_scaled_fi_train[:,:15], y) 

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
          weights='uniform')

In [180]:
predicted=neigh.predict(X_scaled_fi[:,:15])

In [181]:
predicted.shape

(9914,)

In [166]:
cd all

/Users/nathantoubiana/Documents/Codes/Kaggle/all


In [182]:
df_test['preds']=predicted
df_test_keys=df_test[['key','preds']]
df_final=pd.read_csv('sample_submission.csv')
df_final.fare_amount=df_final.merge(df_test_keys,on='key')['preds']
df_final[['key','fare_amount']].to_csv('results_nathan.csv' ,  index=False)